In [4]:
%load_ext autoreload
%autoreload 2

import time
import pandas as pd
import numpy as np
from src.utils.tsp_loader import load_tsp_matrix
from src.algorithms.ihc_numba import solve_tsp
from src.utils.result_saver import save_experiment_results

# ==============================================================
# EKSPERYMENT DLA ALGORYTMU ITERATIVE HILL CLIMBING (IHC)
# ==============================================================
# Sprawdza wpływ parametrów:
#   - n_starts
#   - max_iter
#   - stop_no_improve
#   - neighborhood_type
# ==============================================================

TSP_FILES = ["Dane_TSP_48.xlsx"]
# PARAM_GRID = {
#     "n_starts": [5, 10, 20, 50], # liczba losowych restartów wspinaczki
#     "max_iter": [100, 500, 1000], # maksymalna liczba iteracji jednej wspinaczki
#     "stop_no_improve": [10, 50, 100], # ile kroków bez poprawy kończy wspinaczkę
#     # "neighborhood_type": ["swap", "insert", "two_opt"],
#     "neighborhood_type": ["two_opt"],
# }
PARAM_GRID = {
    "n_starts": [800], # liczba losowych restartów wspinaczki
    "max_iter": [6000], # maksymalna liczba iteracji jednej wspinaczki
    "stop_no_improve": [1200], # ile kroków bez poprawy kończy wspinaczkę
    "neighborhood_type": ["two_opt"],
}
REPEATS = 5
results = []

print("🧠 Warm-up Numba (kompilacja JIT)...")
D = load_tsp_matrix(TSP_FILES[0])
_ = solve_tsp(D, {"n_starts": 2})
print("✅ Kompilacja zakończona. Start eksperymentu...\n")

for tsp_file in TSP_FILES:
    print(f"\n📂 Instancja: {tsp_file}")
    D = load_tsp_matrix(tsp_file)

    for n_starts in PARAM_GRID["n_starts"]:
        for max_iter in PARAM_GRID["max_iter"]:
            for stop_no_improve in PARAM_GRID["stop_no_improve"]:
                for neighborhood_type in PARAM_GRID["neighborhood_type"]:
                    costs, runtimes = [], []

                    for _ in range(REPEATS):
                        params = {
                            "n_starts": n_starts,
                            "max_iter": max_iter,
                            "stop_no_improve": stop_no_improve,
                            "neighborhood_type": neighborhood_type,
                        }
                        start_t = time.perf_counter()
                        route, cost, runtime, meta = solve_tsp(D, params)
                        total_time = time.perf_counter() - start_t

                        costs.append(cost)
                        runtimes.append(total_time)

                    results.append({
                        "instance": tsp_file,
                        "n_starts": n_starts,
                        "max_iter": max_iter,
                        "stop_no_improve": stop_no_improve,
                        "neighborhood_type": neighborhood_type,
                        "mean_cost": round(np.mean(costs), 3),
                        "min_cost": round(np.min(costs), 3),
                        "mean_runtime": np.mean(runtimes),
                    })

# --- zapis wyników ---
df = pd.DataFrame(results)
save_experiment_results(df, filename="ihc_experiment_results.csv", subfolder="IHC")

# --- raport ---
print("\n📊 Podsumowanie wyników:")
print(df.groupby("neighborhood_type")[["mean_cost", "min_cost", "mean_runtime"]].mean().round(3))

best = df.loc[df["mean_cost"].idxmin()]
print(f"\n🏆 Najlepsze parametry: {best.to_dict()}")
print("\n✅ Eksperyment zakończony pomyślnie.")


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
🧠 Warm-up Numba (kompilacja JIT)...
✅ Kompilacja zakończona. Start eksperymentu...


📂 Instancja: Dane_TSP_48.xlsx

📊 Podsumowanie (pierwsze 10 wierszy):
        instance  n_starts  max_iter  stop_no_improve neighborhood_type  mean_cost  min_cost  mean_runtime
Dane_TSP_48.xlsx       800      6000             1200           two_opt     9883.2    9809.0      1.598655

📊 Podsumowanie wyników:
                   mean_cost  min_cost  mean_runtime
neighborhood_type                                   
two_opt               9883.2    9809.0         1.599

🏆 Najlepsze parametry: {'instance': 'Dane_TSP_48.xlsx', 'n_starts': 800, 'max_iter': 6000, 'stop_no_improve': 1200, 'neighborhood_type': 'two_opt', 'mean_cost': 9883.2, 'min_cost': 9809.0, 'mean_runtime': 1.598655008400965}

✅ Eksperyment zakończony pomyślnie.
